In [2]:
from Crypto.Util.number import *
from random import *
from tqdm import *
from gmpy2 import iroot
from math import prod
from sympy.ntheory.modular import crt
from multiprocessing import Pool
from Crypto.Cipher import AES
from hashlib import md5


#part1 get Jacobian's order
extend = 180
F = GF(2^extend, 'z', modulus = x^180+x^3+1)
z = F.gens()[0]
PRx = PolynomialRing(F,"x")
x = PRx.gens()[0]
h = x^2 + 1
f = x^5 + x^2 + 1
H = HyperellipticCurve(f, h)
J = H.jacobian()

C = H.change_ring(GF(2))
p = C.frobenius_polynomial()
alpha = p.roots(ring=QQbar)
N = ZZ(prod(1-a[0]^extend for a in alpha))


#part2 choose a random point G
def gen_point():
    while(1):
        try:
            a = F.from_integer(randint(1,2^extend-1))
            PRy.<y> = PolynomialRing(F,"y")
            y = PRy.gens()[0]
            curve = y^2 + h(a)*y - f(a)
            y , k = curve.roots()[0]
            if(k == 1):
                return J(H(a , y))
        except:
            pass
G = gen_point()


#part3 dlp(remove big primes to use Pohlig-Hellman)
def bsgs(operation):
    P,Q,ord = operation

    #subgroup
    O = J(0)
    P = (N // ord) * P
    Q = (N // ord) * Q

    #Q = kP -> Q = (ax+b)P -> Q-bP = x(aP)
    a = iroot(ord,2)[0]+1
    x = O

    dic = {}
    for i in range(a):
        dic[str(Q-x)] = i
        x += P
    
    x = O
    aP = a*P
    for i in range(a):
        if str(x) in dic.keys():
            d = int(i*a+dic[str(x)])
            return (d,ord)
        x += aP

#Q = kP
def Pohlig_Hellman(P , Q , modlist):
    operation_list = [(P,Q,i) for i in modlist]

    with Pool(16) as pool:
        r = list(pool.imap(bsgs, operation_list))
    
    c = [i[0] for i in r]
    mod = [i[1] for i in r]

    return int(crt(mod,c)[0])

modlist = [101 , 157 , 271 , 461 , 523 , 701 , 829 , 1531 , 1747 , 20341 , 57901 , 71161 , 623881]
big_factors = N // prod(modlist)
G = big_factors * G

#data
s = 
points = 
c = 


s = J(s)
s = big_factors * s
B = Pohlig_Hellman(G,s,modlist)
assert B * G == s

A = []
if(1):
    for i in trange(len(points)):
        point = big_factors * J(points[i])
        d = Pohlig_Hellman(G,point,modlist)
        A.append(d)
        assert d * G == point


#part4 remove redundancy
temp = []
for i in range(len(A)):
    if(i % 8 == 0):
        B -= A[i]
        B %= prod(modlist)
        continue
    temp.append(A[i])
A = temp


#part5 knapsack(use BKZ)
L = Matrix(ZZ,len(A)+2,len(A)+2)
for i in range(len(A)):
    L[i,i] = 2
    L[-2,i] = -1
    L[i,-1] = A[i]
L[-2,-2] = 1
L[-2,-1] = -B
L[-1,-1] = prod(modlist)

L[:,-1:] *= prod(modlist)

bag = []
res = L.BKZ()[0]
vec = vector(ZZ,res[:-2])
if(res[-2] == -1):
    vec = - vec
for i in vec:
    bag.append((i + 1) // 2)


#part6 recover password
password = ""
for i in range(len(bag)):
    if(len(password) % 8 == 0):
        password += "1"
    password += str(bag[i])
password = long_to_bytes(int(password,2))


#part7 decrypt AES
key = md5(password).digest()
enc = AES.new(key,mode = AES.MODE_ECB)
flag = enc.decrypt(c)
print(flag)


#NKCTF{Kn@ps4ck_0F_HyperElliptiCCurve!}

100%|███████████████████████████████████████████████████████████████████████████████████| 80/80 [46:52<00:00, 35.16s/it]


b'NKCTF{Kn@ps4ck_0F_HyperElliptiCCurve!}\n\n\n\n\n\n\n\n\n\n'
